In [2]:
#PREPROCESS_hypergeometric_test_for_metabolomics      22.04.14
#make data table that is ready for hypergeometric test for metabolomics
#seperately done for up, down

import pandas as pd
import numpy as np

In [31]:
def make_subpathway_dict(data_file):
    
    pathway_info_dict = {}
    chem_pathway_info_dict = {}
    
    data_df = pd.read_csv(chem_info_file, sep=',')
    r, c = data_df.shape
    chem_list = []
    
    for i in range(r):
        super_pathway = data_df["SUPER_PATHWAY"][i]
        sub_pathway = data_df["SUB_PATHWAY"][i]        
        chem_name = data_df["CHEMICAL_NAME"][i]
            
        #handle feature name differences
        chem_name = chem_name.replace(',','_')
        chem_name = chem_name.replace('-','_')
        chem_name = chem_name.replace(' ','_')
        chem_name = chem_name.replace("'",'prime')
        try: 
            sub_pathway = sub_pathway.replace('_','')
            pathway = '[%s] %s' % (super_pathway, sub_pathway)
            if chem_name not in chem_list:
                chem_list.append(chem_name)
        except AttributeError: 
            sub_pathway = 'none'
            pathway = '[%s] %s' % (super_pathway, sub_pathway)

        try: pathway_info_dict[pathway].append(chem_name)
        except KeyError: pathway_info_dict[pathway] = [chem_name]

        chem_pathway_info_dict[chem_name] = pathway
    
    return pathway_info_dict, chem_pathway_info_dict, chem_list

def get_feature_list(data_file):
    
    feature_list = []
    open_data = open(data_file,'r')
    data_readlines = open_data.readlines()
    for i in range(len(data_readlines)):
        read = data_readlines[i]
        read = read.replace('\n','')
        feature_list.append(read)
        
    return feature_list

def make_hypergeometric_ready_matrix(feature_list, chem_pathway_info_dict, pathway_info_dict, chem_list, output_file):
    
    num_sig_feature = len(feature_list)
    pathway_dict = {}
    mentioned_pathway_list = []
    #summarize counts
    for feature in feature_list:
        pathway = chem_pathway_info_dict[feature]
        if pathway != "[nan] none":
            try: 
                pathway_dict[pathway][0] += 1
                pathway_dict[pathway][1].append(feature)
            except KeyError: 
                pathway_dict[pathway] = [1, [feature]]

            if pathway not in mentioned_pathway_list:
                mentioned_pathway_list.append(pathway)
    
    output_txt = open(output_file, 'w')
    output_txt.write('pathway\tnumber_of_metabolites_in_pathway(m)\tsignificant_metabolites_in_pathway(x)\ttotal_number_of_metabolites(n)\tdetected_significant_metabolites(k)\telements\n')
    for pathway in mentioned_pathway_list:
        m = len(pathway_info_dict[pathway])
        k = pathway_dict[pathway][0]
        
        output_txt.write('%s\t%s\t%s\t%s\t%s\t' % (pathway, m, k, len(chem_list), num_sig_feature))
        for elements in pathway_dict[pathway][1]:
            output_txt.write('%s,' % elements)
        
        output_txt.write('\n')


In [32]:
metabolomic_file_dir = '../../../../../analysis/full_data/cytoscape/gse'
chem_info_file = '../../../../../raw_data/RA_metabolon/MAYO-03-21ML+/RA_ACPA_metabolomics_chem_info.csv'

pathway_info_dict, chem_pathway_info_dict, chem_list = make_subpathway_dict(chem_info_file)

In [33]:
data_file = '%s/network_metabolite_list.tsv' % (metabolomic_file_dir)
output_file = '%s/metabolomics.hypergeometric.ready.tsv' % (metabolomic_file_dir)

feature_list = get_feature_list(data_file)

make_hypergeometric_ready_matrix(feature_list, chem_pathway_info_dict, pathway_info_dict, chem_list, output_file)


In [14]:
feature_list

['X_24295',
 'X_23636',
 'X_12462',
 'tartronate_(hydroxymalonate)',
 'sphingosine_1_phosphate',
 'sarcosine',
 'pyruvate',
 'oxindolylalanine',
 'ornithine',
 'lactate',
 'cys_gly__oxidized',
 '4_methyl_2_oxopentanoate',
 '3_5_dichloro_2_6_dihydroxybenzoic_acid',
 '2prime_O_methylcytidine']